__собираем предложения о продаже домов__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# каталог с файлами объявлений
avito_raw_data_path = 'data/raw/house/avito' 
# cian_raw_data_path = 'data/raw/cian'
result_file_path = 'data/data_house.pkl'

In [5]:
# ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
# ts

## собираем данные авито.ру

In [6]:
import re
from os import listdir

raw_data_files = sorted([
        avito_raw_data_path+'/'+f 
        for f in listdir(avito_raw_data_path) 
        if re.match(r'.+\.xlsx$',f)
    ])
raw_data_files

['data/raw/house/avito/avito_2022-09-15_11-54_raw_sevastopol.xlsx',
 'data/raw/house/avito/avito_2022-09-15_14-42_raw_bahchisaray.xlsx',
 'data/raw/house/avito/avito_2022-09-15_14-47_raw_sudak.xlsx',
 'data/raw/house/avito/avito_2022-09-16_15-21_raw_sevastopol.xlsx',
 'data/raw/house/avito/avito_2022-09-16_15-22_raw_bahchisaray.xlsx',
 'data/raw/house/avito/avito_2022-09-16_15-24_raw_sudak.xlsx']

In [7]:
data_avito = pd.concat([
        pd
        .read_excel(f)
        #.dropna()
        .drop_duplicates(['avito_id','description'])
        .rename(columns={'page':'avito_page'})
        .reset_index(drop=True)  
        for f in raw_data_files
    ]).reset_index(drop=True)

print(len(data_avito))

5588


In [8]:
# data_avito

In [9]:
from lib.avito import AvitoDataCleanerRealtyHouse

data_avito = AvitoDataCleanerRealtyHouse().transform( data_avito )

print(len(data_avito))

5588


In [10]:
data_avito.sample(3)

,avito_id,title,price,obj_name,adr,description,avito_page,ts,place,house_area,is_part,is_townhouse,is_SNT,land_area,priceM,land_size_category,house_size_category
1965,2546974930,дом 102 м² на участке 3 сот.,13500000,,"Мирный пер., 11","Продаётся новый дом в тихом уютном месте рядом с центром города. На 1 этаже просторная гостиная, кухня и санузел. На 2 этаже три спальни с ванной комнатой. Участок Ижс площадью 3 сотки. \nВ доме все необходимые коммуникации: свет, вода, газ, канализация (бо",41,2022-09-15 11:53:29.976,sevastopol,102.00,False,False,False,3.00,13.50,2-4,70-150
3157,2512781371,"дом 120 м² на участке 4,7 сот.",8300000,,Монастырское ш.,"Продаю дом. Дом из ракушки, свет, вода, септик. Находится на Фиоленте. В 5 минутах пешей доступности от Монастыря( Яшмовый пляж) СТ Автомобилист 250. Я хозяин.",7,2022-09-16 15:21:10.419,sevastopol,120.00,False,False,False,4.70,8.30,4-8,70-150
3111,2501346797,"дом 44 м² на участке 4,5 сот.",10000000,,СТ Море,"Жилой дом с пропиской! Фото, видео по запросу скину! Рядом пляж (50 м), инфраструктура: ТЦ Метро — 3 км, несколько местных супермаркетов тоже рядом, школа гимназия — 3 км(школьный автобус), несколько детсадов рядом. Продуктовый магазин — 500 м. Асфальтиров",7,2022-09-16 15:21:10.419,sevastopol,44.00,False,False,True,4.50,10.00,4-8,30-50


In [11]:
data_avito['place'].unique()

array(['sevastopol', 'bahchisaray', 'sudak'], dtype=object)

In [12]:
assert len(data_avito)>0

In [13]:
data_avito.info(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5588 entries, 0 to 5587
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   avito_id             5588 non-null   int64         
 1   title                5588 non-null   object        
 2   price                5588 non-null   int64         
 3   obj_name             5588 non-null   object        
 4   adr                  5588 non-null   object        
 5   description          5588 non-null   object        
 6   avito_page           5588 non-null   int64         
 7   ts                   5588 non-null   datetime64[ns]
 8   place                5588 non-null   object        
 9   house_area           5588 non-null   float64       
 10  is_part              5588 non-null   bool          
 11  is_townhouse         5588 non-null   bool          
 12  is_SNT               5588 non-null   bool          
 13  land_area            5588 non-nul

In [14]:
data_avito.isnull().any()

avito_id               False
title                  False
price                  False
obj_name               False
adr                    False
description            False
avito_page             False
ts                     False
place                  False
house_area             False
is_part                False
is_townhouse           False
is_SNT                 False
land_area              False
priceM                 False
land_size_category      True
house_size_category     True
dtype: bool

In [15]:
data_avito[ data_avito['land_size_category'].isnull() ]

,avito_id,title,price,obj_name,adr,description,avito_page,ts,place,house_area,is_part,is_townhouse,is_SNT,land_area,priceM,land_size_category,house_size_category
2283,2337799298,,11900000,,Восточная ул.,"Предлагаю 1/2 дома общей площадью 120 кв. М. В центре города в отличном районе для проживания на земельном участке 1.6 сотки. Вся инфраструктура рядом, транспортные развязки в шаговой доступности. При этом дом расположен в тихом месте, с каждого этажа дома",47,2022-09-15 11:53:29.976,sevastopol,0.00,True,False,False,0.00,11.90,NaN,NaN
2614,2034310938,,5000000,,"с. Танковое, ул. Ходырева","Собственник. Срочно продам дом 74,3 кв м на земельном участке 814 м кв. Санузел и туалет в доме, 2 спальни. 1965 года постройки, камень бут, очень тёплый зимой и прохладный летом. Газ, приборы учета на все коммуникации. В дом есть 2 входа. Также на участке",4,2022-09-15 14:42:27.254,bahchisaray,0.00,False,False,False,0.00,5.00,NaN,NaN
2624,2391096190,,3000000,,"с. Растущее, Садовая ул., 8","Продам Дом 75 КВ. М, 10 Соток Земли Ижс Продам дом в живописном районе Крыма, в селе Растущее 10 соток земли, 75 кв. М, 4 комнаты, коммуникации: свет, газ, вода, скважина, отопление Агв. Дом чистый, уютный, заходи живи. Участок ровный, сад, весь обработанн",4,2022-09-15 14:42:27.254,bahchisaray,0.00,False,False,True,0.00,3.00,NaN,NaN
2733,2418227584,,5500000,,"Платановый квартал, 21",Недострой (Ижс).\nЗемля 5.2 сотки (собственность).\nВсе вопросы по телефону.,1,2022-09-15 14:47:45.056,sudak,0.00,False,False,False,0.00,5.50,NaN,NaN
5076,2337799298,,11900000,,Восточная ул.,"Предлагаю 1/2 дома общей площадью 120 кв. М. В центре города в отличном районе для проживания на земельном участке 1.6 сотки. Вся инфраструктура рядом, транспортные развязки в шаговой доступности. При этом дом расположен в тихом месте, с каждого этажа дома",48,2022-09-16 15:21:10.419,sevastopol,0.00,True,False,False,0.00,11.90,NaN,NaN
5213,2391096190,,3000000,,"с. Растущее, Садовая ул., 8","Продам Дом 75 КВ. М, 10 Соток Земли Ижс Продам дом в живописном районе Крыма, в селе Растущее 10 соток земли, 75 кв. М, 4 комнаты, коммуникации: свет, газ, вода, скважина, отопление Агв. Дом чистый, уютный, заходи живи. Участок ровный, сад, весь обработанн",1,2022-09-16 15:22:23.263,bahchisaray,0.00,False,False,True,0.00,3.00,NaN,NaN
5384,2034310938,,5000000,,"с. Танковое, ул. Ходырева","Собственник. Срочно продам дом 74,3 кв м на земельном участке 814 м кв. Санузел и туалет в доме, 2 спальни. 1965 года постройки, камень бут, очень тёплый зимой и прохладный летом. Газ, приборы учета на все коммуникации. В дом есть 2 входа. Также на участке",4,2022-09-16 15:22:23.263,bahchisaray,0.00,False,False,False,0.00,5.00,NaN,NaN
5511,2418227584,,5500000,,"Платановый квартал, 21",Недострой (Ижс).\nЗемля 5.2 сотки (собственность).\nВсе вопросы по телефону.,1,2022-09-16 15:24:24.472,sudak,0.00,False,False,False,0.00,5.50,NaN,NaN


---

In [16]:
data = data_avito.reset_index(drop=True) 
print(len(data))
assert len(data)>0

5588


## обновляем таблицу адресов

In [17]:
from lib.locator import AddressTransformer

data['adr'] = AddressTransformer().transform( data['adr'] )

places = { # поле 'place' и название города
'bahchisaray': 'Бахчисарай,',
      'sudak': 'Судак,',
 'sevastopol': 'Севастополь,',
}

for k,v in places.items():
    print(k,v) # докидываем в строку адреса название города если его нет
    f =  (data['place']==k) & (~data['adr'].str.match(v))
    data.loc[f,'adr'] = v +' '+ data.loc[f,'adr']

bahchisaray Бахчисарай,
sudak Судак,
sevastopol Севастополь,


----

In [18]:
from lib.locator import LocationUpdater
from lib.locator import GeocoderYandex
from yandex_keys import keys


loc_file_path = 'data/location.pkl'# таблица адресов

loc = LocationUpdater(
        locator=GeocoderYandex(keys[2]),
    ).transform(
        adr=data['adr'],
        loc=pd.read_pickle(loc_file_path),
        show_pbar=True,
    )

[INFO    ] 2022-09-16 15:33:46 | LocationUpdater: 10055 addresses in location table
[INFO    ] 2022-09-16 15:33:46 | LocationUpdater: 10152 addresses total
[INFO    ] 2022-09-16 15:33:46 | LocationUpdater: 10055 addresses defined
[INFO    ] 2022-09-16 15:33:46 | LocationUpdater: 97 addresses undefined


  0%|          | 0/97 [00:00<?, ?it/s]

[INFO    ] 2022-09-16 15:34:15 | LocationUpdater: 97 new addresses found


In [19]:
loc

,adr,latitude,longitude,truncated
0,"Севастополь, улица Токарева, 7 этап, блок-секция 3",44.78,33.55,False
1,"Севастополь, улица Токарева, 7 этап, блок-секция 1",44.51,33.60,False
2,"Севастополь, улица Генерала Жидилова, 4",44.61,33.52,False
3,"Севастополь, улица Николая Музыки, 96",44.85,33.63,False
4,"Севастополь, улица Лётчиков, д. 10",44.58,33.52,False
...,...,...,...,...
10147,"Бахчисарай, с. Фурмановка, ул. Будённого, 47",44.76,33.74,NaN
10148,"Судак, СНТ Солнечная Долина, Морская ул.",44.84,35.05,NaN
10149,"Судак, ул. Десантников, 6",44.86,34.98,NaN
10150,"Судак, Гвардейская ул.",44.86,34.97,NaN


In [20]:
loc.sample(3)

,adr,latitude,longitude,truncated
5401,"улица Челнокова, 13/31",44.59,33.43,False
4754,"Парковая улица , 14Б",44.56,33.53,False
1591,"Севастополь, проспект Генерала Острякова, 170А",44.56,33.52,False


In [21]:
assert len(loc)>0
loc.to_pickle(loc_file_path)

## дополняем данные геометкой

In [22]:
data = data.merge(loc[['adr','latitude','longitude',]],on=['adr'],how='left')

print('всего записей:', len( data) )
print('записей без геометки:', len( data[ data['latitude'].isnull() ] ) )

всего записей: 5599
записей без геометки: 0


In [23]:
assert len(data)>0
data.to_pickle(result_file_path)

In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5599 entries, 0 to 5598
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   avito_id             5599 non-null   int64         
 1   title                5599 non-null   object        
 2   price                5599 non-null   int64         
 3   obj_name             5599 non-null   object        
 4   adr                  5599 non-null   object        
 5   description          5599 non-null   object        
 6   avito_page           5599 non-null   int64         
 7   ts                   5599 non-null   datetime64[ns]
 8   place                5599 non-null   object        
 9   house_area           5599 non-null   float64       
 10  is_part              5599 non-null   bool          
 11  is_townhouse         5599 non-null   bool          
 12  is_SNT               5599 non-null   bool          
 13  land_area            5599 non-nul